In [ ]:
def parse_entry(entry):
    op=int(entry[0])
    entry=entry[2:]
    if op == 1:
        entry=entry.split(';')
        return [1,{'time':int(entry[0]),'log_type':entry[1],
                'severity':float(entry[2])}]
    entry=entry.split(' ')
    if op == 2:
        return [2,{'log_type':entry[0]}]
    if op == 3:
        return [3,{'action':entry[0],'time':int(entry[1])}]
    if op == 4:
        return [4,{'action':entry[0],'log_type':entry[1],'time':int(entry[2])}]

In [ ]:
def sol(i,entry_dict):
    op,parsed=parse_entry(i)
    if op == 1:
        entry_dict[(parsed['time'],parsed['log_type'])]=parsed['severity']
        return entry_dict
    if op == 2:
        sev_arr=[]
        log_type=parsed['log_type']
        for keys,val in entry_dict.items():
            if keys[1]==log_type:
                sev_arr.append(val)
        if not sev_arr:
                sev_arr=[0.0]
        return([min(sev_arr),max(sev_arr),sum(sev_arr)/len(sev_arr)])
    if op == 3:
        sev_arr=[]
        time=parsed['time']
        if parsed['action']=='BEFORE':
            for keys,val in entry_dict.items():
                if keys[0]<time:
                    sev_arr.append(val)
            if not sev_arr:
                sev_arr=[0.0]
            return([min(sev_arr),max(sev_arr),sum(sev_arr)/len(sev_arr)])
        else:
            for keys,val in entry_dict.items():
                if keys[0]>time:
                    sev_arr.append(val)
            if not sev_arr:
                sev_arr=[0.0]
            return([min(sev_arr),max(sev_arr),sum(sev_arr)/len(sev_arr)])
    if op == 4:
        sev_arr=[]
        time=parsed['time']
        log_type=parsed['log_type']
        if parsed['action']=='BEFORE':
            for keys,val in entry_dict.items():
                if keys[0]<time and keys[1]==log_type:
                    sev_arr.append(val)
            if not sev_arr:
                sev_arr=[0.0]
            return([min(sev_arr),max(sev_arr),sum(sev_arr)/len(sev_arr)])
        else:
            for keys,val in entry_dict.items():
                if keys[0]>time and keys[1]==log_type:
                    sev_arr.append(val)
            if not sev_arr:
                sev_arr=[0.0]
            return([min(sev_arr),max(sev_arr),sum(sev_arr)/len(sev_arr)])

In [ ]:
import random

def generate_test_input(num_entries):
    types = ['ERROR', 'WARNING', 'INFO', 'DEBUG', 'CRITICAL', 'ALERT', 
             'NOTICE', 'EMERGENCY', 'TRACE', 'FATAL', 'SECURITY', 'AUDIT', 
             'INFO', 'PERFORMANCE', 'CONFIG']
    timestamp = 1 
    with open('input.txt', 'w') as file:
        for _ in range(30):
            log_type = random.choice(types) 
            timestamp += random.randint(1,5) 
            severity = round(random.uniform(1e-6,1e6),2)
            file.write(f"1 {timestamp};{log_type};{severity}\n")
        for _ in range(num_entries):
            choice = random.choices([1, 2, 3, 4], weights=[0.4, 0.3, 0.15, 0.15])[0]
            if choice == 1:
                log_type = random.choice(types) 
                severity = round(random.uniform(1e-6,1e6),2)
                timestamp += random.randint(1,5)
                file.write(f"1 {timestamp};{log_type};{severity}\n")
            elif choice == 2:
                log_type = random.choice(types) 
                file.write(f"2 {log_type}\n")
            elif choice == 3:
                subcommand = random.choice(['BEFORE', 'AFTER'])
                timestamp += random.randint(1,5)
                file.write(f"3 {subcommand} {timestamp-30}\n")
            elif choice == 4:
                subcommand = random.choice(['BEFORE', 'AFTER'])
                log_type = random.choice(types)  
                timestamp += random.randint(1,5)
                file.write(f"4 {subcommand} {log_type} {timestamp-30}\n") 
if __name__ == "__main__":
    generate_test_input(300)

In [ ]:
def final_sol(in_file_path,out_file_path='output.txt'):
    with open(in_file_path, 'r') as input_file:
        input_lines = [s[:-1] for s in input_file.readlines()]
    output=[]
    entry_dict={}
    for i in input_lines:
        if i[0]=='1':
            entry_dict=sol(i,entry_dict)
        else:
            output.append(sol(i,entry_dict))
    with open(out_file_path, 'w') as output_file:
        for result in output:
            output_file.write(f"Min: {result[0]}, Max: {result[1]}, Mean: {round(result[2],6)}\n")

In [211]:
#EXAMPLE:

final_sol('/kaggle/working/input.txt','/kaggle/working/output.txt')